In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sp
import scipy.linalg as la
%matplotlib inline
plt.rcParams['figure.figsize'] = 15,10

In [8]:
image_data = pd.read_csv("./pokemon_image.csv")
image_data.head()

,Name,Type1,Type2
0,bulbasaur,Grass,Poison
1,ivysaur,Grass,Poison
2,venusaur,Grass,Poison
3,charmander,Fire,NaN
4,charmeleon,Fire,NaN


In [9]:
stats_data = pd.read_csv("./pokemon_stats.csv")
stats_data.head()

,abilities,against_bug,against_dark,against_dragon,against_electric,against_fairy,against_fight,against_fire,against_flying,against_ghost,...,percentage_male,pokedex_number,sp_attack,sp_defense,speed,type1,type2,weight_kg,generation,is_legendary
0,"['Overgrow', 'Chlorophyll']",1.0,1.0,1.0,0.5,0.5,0.5,2.0,2.0,1.0,...,88.1,1,65,65,45,grass,poison,6.9,1,0
1,"['Overgrow', 'Chlorophyll']",1.0,1.0,1.0,0.5,0.5,0.5,2.0,2.0,1.0,...,88.1,2,80,80,60,grass,poison,13.0,1,0
2,"['Overgrow', 'Chlorophyll']",1.0,1.0,1.0,0.5,0.5,0.5,2.0,2.0,1.0,...,88.1,3,122,120,80,grass,poison,100.0,1,0
3,"['Blaze', 'Solar Power']",0.5,1.0,1.0,1.0,0.5,1.0,0.5,1.0,1.0,...,88.1,4,60,50,65,fire,NaN,8.5,1,0
4,"['Blaze', 'Solar Power']",0.5,1.0,1.0,1.0,0.5,1.0,0.5,1.0,1.0,...,88.1,5,80,65,80,fire,NaN,19.0,1,0


# Data Preparation: Getting the top 3 colors per image 

In [41]:
import imageio
import os
from colorthief import ColorThief

In [ ]:
# CODE FOR PLOTTING 
#     construct the color compressed version of X
#     X_color_compressed = np.zeros(X.shape)
#     X_color_compressed = centers[labels,:]

#     # reshape the color compressed version of X
#     X_ccompressed = X_color_compressed.reshape((r,c,3))

#     # show the pictures side by side
#     # Two subplots, unpack the axes array immediately
#     f, (ax1, ax2) = plt.subplots(1, 2, sharey=True)
#     ax1.imshow(pokemon)
#     ax2.imshow(X_ccompressed)

In [78]:
directory = os.fsencode("./images")

color1 = []
color2 = []
color3 = []
names = []

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".png"): 
        full_filename = "./images/" + filename
        names.append(filename[:-4])
        color_thief = ColorThief(full_filename)
        palette = color_thief.get_palette(color_count=3)
        color1.append(palette[0])
        color2.append(palette[1])
        color3.append(palette[2])
    else:
        continue 

In [79]:
colors = pd.DataFrame({"pokemon":names, "color1":color1, "color2":color2, "color3":color3})
colors.head()

,pokemon,color1,color2,color3
0,parasect,"(225, 173, 86)","(94, 63, 41)","(250, 198, 233)"
1,lumineon,"(55, 79, 86)","(189, 234, 231)","(180, 141, 163)"
2,raikou,"(195, 173, 90)","(60, 54, 45)","(219, 201, 211)"
3,dedenne,"(196, 162, 87)","(40, 38, 34)","(218, 215, 188)"
4,pyroar,"(198, 96, 71)","(243, 205, 131)","(84, 76, 69)"


In [81]:
import webcolors

# Source: https://stackoverflow.com/questions/9694165/convert-rgb-color-to-english-color-name-like-green-with-python

def closest_colour(requested_colour):
    min_colours = {}
    for key, name in webcolors.css3_hex_to_names.items():
        r_c, g_c, b_c = webcolors.hex_to_rgb(key)
        rd = (r_c - requested_colour[0]) ** 2
        gd = (g_c - requested_colour[1]) ** 2
        bd = (b_c - requested_colour[2]) ** 2
        min_colours[(rd + gd + bd)] = name
    return min_colours[min(min_colours.keys())]

def get_colour_name(requested_colour):
    try:
        closest_name = actual_name = webcolors.rgb_to_name(requested_colour)
    except ValueError:
        closest_name = closest_colour(requested_colour)
        actual_name = None
    return actual_name, closest_name

# requested_colour = (119, 172, 152)
# actual_name, closest_name = get_colour_name(requested_colour)

# print "Actual colour name:", actual_name, ", closest colour name:", closest_name

In [90]:
colors['webcolor1'] = colors['color1'].apply(lambda row: get_colour_name(row)[1])
colors['webcolor2'] = colors['color2'].apply(lambda row: get_colour_name(row)[1])
colors['webcolor3'] = colors['color3'].apply(lambda row: get_colour_name(row)[1])
colors.head()

,pokemon,color1,color2,color3,webcolor1,webcolor2,webcolor3
0,parasect,"(225, 173, 86)","(94, 63, 41)","(250, 198, 233)",sandybrown,darkolivegreen,pink
1,lumineon,"(55, 79, 86)","(189, 234, 231)","(180, 141, 163)",darkslategrey,paleturquoise,rosybrown
2,raikou,"(195, 173, 90)","(60, 54, 45)","(219, 201, 211)",darkkhaki,darkslategrey,thistle
3,dedenne,"(196, 162, 87)","(40, 38, 34)","(218, 215, 188)",darkkhaki,darkslategrey,lightgrey
4,pyroar,"(198, 96, 71)","(243, 205, 131)","(84, 76, 69)",indianred,khaki,darkolivegreen


In [101]:
color_categories = pd.read_csv("./colors.csv")
color_categories.head()

,category,webcolor
0,pink,pink
1,pink,lightpink
2,pink,hotpink
3,pink,deeppink
4,pink,palevioletred


In [114]:
color_categories['webcolor'] = color_categories['webcolor'].astype(str)

In [119]:
for item in color_categories['webcolor']:
    print(item)
    print(item == 'pink')
    print(type(item))

pink 
False
<class 'str'>
lightpink
False
<class 'str'>
hotpink 
False
<class 'str'>
deeppink
False
<class 'str'>
palevioletred
False
<class 'str'>
mediumvioletred
False
<class 'str'>
lavendar 
False
<class 'str'>
thistle 
False
<class 'str'>
plum
False
<class 'str'>
orchid 
False
<class 'str'>
violet 
False
<class 'str'>
fuchsia 
False
<class 'str'>
magenta
False
<class 'str'>
mediumorchid 
False
<class 'str'>
darkorchid 
False
<class 'str'>
darkviolet 
False
<class 'str'>
blueviolet 
False
<class 'str'>
darkmagenta 
False
<class 'str'>
purple 
False
<class 'str'>
mediumpurple 
False
<class 'str'>
mediumslateblue 
False
<class 'str'>
slateblue 
False
<class 'str'>
darkslateblue 
False
<class 'str'>
rebeccarpurple 
False
<class 'str'>
indigo 
False
<class 'str'>
lightsalmon 
False
<class 'str'>
salmon 
False
<class 'str'>
darksalmon 
False
<class 'str'>
lightcoral 
False
<class 'str'>
indianred 
False
<class 'str'>
crimson 
False
<class 'str'>
red 
False
<class 'str'>
firebrick 
False


In [ ]:
colors['category1'] = colors['webcolor1'].apply(lambda row: get_colour_name(row)[1])
colors['category2'] = colors['webcolor2'].apply(lambda row: get_colour_name(row)[1])
colors['category3'] = colors['webcolor3'].apply(lambda row: get_colour_name(row)[1])
colors.head()